In [11]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = (10, 8)
import collections

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from scipy.stats import entropy

Часть 1. Игрушечный набор данных "девушка в баре"
Цель – "на пальцах", с помощью игрушечной задачи классификации разобраться в том, как работают деревья решений. Само по себе дерево решений – довольно слабый алгоритм, но основанные на нем алгоритмы случайного леса и градиентного бустинга - пожалуй, лучшее, что есть на сегодняшний день (в задачах, где можно обойтись без нейронных сетей). Поэтому разобраться в том, как работает дерево решений, полезно.

Рассмотрим игрушечную задачу бинарной классификации: поедет ли с Вами девушка из бара? Это будет зависеть от Вашей внешности и красноречия, крепости предлагаемых напитков и, как это ни меркантильно, от количества потраченных в баре денег.

In [2]:
# Создание датафрейма с dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [3]:
features = ['Внешность', 'Алкоголь_в_напитке',
            'Уровень_красноречия', 'Потраченные_деньги']

#### Обучающая выборка

In [49]:
df_train = {}
df_train['Внешность'] = ['приятная', 'приятная', 'приятная', 'отталкивающая',
                         'отталкивающая', 'отталкивающая', 'приятная'] 
df_train['Алкоголь_в_напитке'] = ['да', 'да', 'нет', 'нет', 'да', 'да', 'да']
df_train['Уровень_красноречия'] = ['высокий', 'низкий', 'средний', 'средний', 'низкий',
                                   'высокий', 'средний']
df_train['Потраченные_деньги'] = ['много', 'мало', 'много', 'мало', 'много',
                                  'много', 'много']
df_train['Поедет'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

df_train = create_df(df_train, features)
df_train

,Поедет,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_низкий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,False,True,True,False,True,False,False,False,True
1,1,False,True,True,False,False,True,False,True,False
2,0,False,True,False,True,False,False,True,False,True
3,1,True,False,False,True,False,False,True,True,False
4,1,True,False,True,False,False,True,False,False,True
5,0,True,False,True,False,True,False,False,False,True
6,0,False,True,True,False,False,False,True,False,True


#### Тестовая выборка

In [5]:
df_test = {}
df_test['Внешность'] = ['приятная', 'приятная', 'отталкивающая'] 
df_test['Алкоголь_в_напитке'] = ['нет', 'да', 'да']
df_test['Уровень_красноречия'] = ['средний', 'высокий', 'средний']
df_test['Потраченные_деньги'] = ['много', 'мало', 'много']
df_test = create_df(df_test, features)
df_test

,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,False,True,False,True,False,True,False,True
1,False,True,True,False,True,False,True,False
2,True,False,True,False,False,True,False,True


In [6]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y = df_train['Поедет']
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Потраченные_деньги_мало,Внешность_отталкивающая,Алкоголь_в_напитке_да,Внешность_приятная,Уровень_красноречия_средний,Потраченные_деньги_много
0,False,True,False,False,True,True,False,True
1,False,False,True,False,True,True,False,False
2,True,False,False,False,False,True,True,True
3,True,False,True,True,False,False,True,False
4,False,False,False,True,True,False,False,True
5,False,True,False,True,True,False,False,True
6,False,False,False,False,True,True,True,True


In [7]:
df_test

,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Потраченные_деньги_мало,Внешность_отталкивающая,Алкоголь_в_напитке_да,Внешность_приятная,Уровень_красноречия_средний,Потраченные_деньги_много
0,True,False,False,False,False,True,True,True
1,False,True,True,False,True,True,False,False
2,False,False,False,True,True,False,True,True


Постройте от руки (или в графическом редакторе) дерево решений для этого набора данных. Дополнительно (для желающих) – можете сделать отрисовку дерева и написать код для построения всего дерева.

Вопрос 1. Какова энтропия начальной системы (S0
)? Под состояниями системы понимаем значения признака "Поедет" – 0 или 1 (то есть всего 2 состояния).

Вопрос 2. Рассмотрим разбиение обучающей выборки по признаку "Внешность_приятная". Какова энтропия S1
 левой группы, тех, у кого внешность приятная, и правой группы – S2
? Каков прирост информации при данном разбиении (IG)?


In [31]:
entropy(y)

1.0986122886681096

In [44]:
def shannon_entropy(data):
    # Подсчет количества каждого значения
    _, counts = np.unique(data, return_counts=True)
    
    # Вычисление вероятностей появления каждого значения
    probabilities = counts / len(data)
    
    # Вычисление энтропии по формуле Шеннона
    entropy = -np.sum(probabilities * np.log2(probabilities))
    
    return entropyt

In [90]:
S0 = shannon_entropy(y)

In [89]:
S2 = shannon_entropy(pd.concat([y, df_train['Внешность_приятная']], axis=1)
                .query('Внешность_приятная == False')['Поедет'])
print(S2.round(3))
S1 = shannon_entropy(pd.concat([y, df_train['Внешность_приятная']], axis=1)
                .query('Внешность_приятная == True')['Поедет'])


0.918
0.811


In [96]:
(S0 - 4/7*S1 - 3/7*S2).round(3)

0.128

Постройте с помощью sklearn дерево решений, обучив его на обучающей выборке. Глубину можно не ограничивать.